## Automated Speaker Verification - A Study in Deep Neural Networks and Transfer Learning 

*"Identity theft is not a joke" - Dwight Shrute, The Office*

Automated speaker verification (ASV), in the modern day, is omnipresent in smart devices and in services offered by call centres. It serves as a biometric means of authenticating a claimed identity by analysing a spoken sample of their voice. 
our team sought to extend our understanding of neural style transfer by exploring how convolutional neural networks could be used to harness the ability to speak in the voice of another person.

#### Requirements

To activate a virtual environment, run

```shell
python3 -m venv env
source env/bin/activate
```

To install the required python packages, run

```shell
pip install -r requirements.txt
```

#### Resources

https://arxiv.org/pdf/2005.07143.pdf

https://www.deepmind.com/blog/wavenet-a-generative-model-for-raw-audio

https://becominghuman.ai/convoice-real-time-zero-shot-voice-style-transfer-with-convolutional-network-4c7b7fff66c9

https://arxiv.org/pdf/1703.10135.pdf

https://arxiv.org/pdf/1905.05879v2.pdf

https://medium.com/@ageitgey/machine-learning-is-fun-part-6-how-to-do-speech-recognition-with-deep-learning-28293c162f7a



#### Transfer Learning

The DNN is trained to classify speakers using a training set of speech recorded from a large number of training speakers. (Talk about Vox-Celeb dataset). To leverage feature representations from the pretrained model on the large dataset, speech recorded from each set of enrollment speakers is passed as input to the trained DNN. This enables the computations of deeper hidden features for each speaker in the enrollment set, which are then averaged to generate a compact deep embedding associated with that speaker.   

#### Zero-Shot Learning



#### Data Preparation

Our pretrained model is trained on audio files collected from the VoxCeleb1 + VoxCeleb2 dataset, consisting of speech samples from over 7000 different speakers of a wide range of ethnicities, accents, professions and ages ([SOURCE](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/)). Achieves an accuracy of approximately 98-99%.

#### Verification through Inference

To verify the identity of an unknown speaker, a test utterance of the unknown speaker is passed as input to the trained DNN. A compact deep embedding associated with the unknown speaker is generated and compared with the compact deep embeddings associated with each of the enrollment speakers through calculation of Cosine Distance Similarity. (Talk about Cosine Distance - include brief background?). The distance between the compared compact deep embeddings corresponds to the likelihood that the unknown speaker belongs to the set of enrolled speakers. 

In [ ]:
# Imports for inference

import os
import shutil
import glob
from random import shuffle
from torch.nn import CosineSimilarity 
from torchaudio import load as load_signal
from speechbrain.pretrained import EncoderClassifier

To enable easy accessibility to the most recently trained model, we first move it to the "content/best_model" path along with associated hyperparameters and class labels. 

In [ ]:
src_path = "results/speaker_id/1986/save/"  # Path to trained network checkpoints
dest_path = "content/best_model/"           # Path to store most recently trained model information 

if os.path.exists(dest_path):
    shutil.rmtree(dest_path)

os.mkdir(dest_path)
shutil.copy2("./hparams_inference.yaml", dest_path)
shutil.copy2(src_path + "label_encoder.txt", dest_path)
ckpt_files = glob.glob(src_path + "CKPT*")
if not ckpt_files:
    print("No trained checkpoints")
    exit(1)
latest_ckpt_path = max(ckpt_files, key=os.path.getctime)
for file in glob.glob(latest_ckpt_path + "/*"):
    shutil.copy2(file, dest_path)

Now, to begin inference, we identify the path to the recorded test signal and the *unique* user id that the test signal should be tested against. << Briefly explain EncoderClassifier class. >>

In [ ]:
# Build Classifier
classifier = EncoderClassifier.from_hparams(source="content/best_model",  hparams_file='hparams_inference.yaml', savedir="content/best_model")

In [ ]:
# Cosine Similarity
similarity = CosineSimilarity(dim=-1, eps=1e-8) # dim=-1 refers to the last dimension (i.e. the embedding dimension)

In [ ]:
! ls

![Calculation of Cosine Similarity]("other/cos_sim.png")

In [ ]:
def extract_audio_embeddings(model, wav_audio_file_path: str) -> tuple:
    """Feature extractor that embeds audio into a vector."""
    signal, _ = load_signal(wav_audio_file_path)  # Reformat audio signal into a tensor
    embeddings = model.encode_batch(
        signal
    )  # Pass tensor through pretrained neural net and extract representation
    return embeddings

def verify(s1, s2):
    global similarity
    score = similarity(s1, s2) # resulting tensor has scores = embedding dimensionality 
    for s in score: 
        if s > 0.25: return True
    return False

In [ ]:
test_emb = extract_audio_embeddings(classifier, test_signal_path)

spk_samples = glob.glob(f"data/user_data/raw/{spk_id}/*/*.wav")
shuffle(spk_samples)
for sample_path in spk_samples[:5]: # test on up to 5 random samples
    print(f"Testing sample against {sample_path}")
    sample_emb = extract_audio_embeddings(classifier, sample_path)
    if verify(test_emb, sample_emb):
        print("User Verified")
        exit(0)

print("Suspicious User - Access Denied")

#### Key Features to Discuss

- Network Architecture (ECAPA-TDNN architecture) (See [here](https://arxiv.org/pdf/2005.07143.pdf))
    - Channel- and context-dependent attention mechanism
    - Multi-layer Feature Aggregation (MFA)
    - AAMsoftmax loss
- Connectionist temporal classification loss (CTC loss)
- VAD
- Statistical pooling
- Data Augmentation (Adding time/frequency dropouts, speed change, environmental corruption, noise addition)
- Dropout
- Normalisation
- Linear Learning Rate Decay and Adam Optimiser



#### Citation

##### Datasets

@InProceedings{Nagrani17,
  author       = "Nagrani, A. and Chung, J.~S. and Zisserman, A.",
  title        = "VoxCeleb: a large-scale speaker identification dataset",
  booktitle    = "INTERSPEECH",
  year         = "2017",
}

@InProceedings{Nagrani17,
  author       = "Chung, J.~S. and Nagrani, A. and Zisserman, A.",
  title        = "VoxCeleb2: Deep Speaker Recognition",
  booktitle    = "INTERSPEECH",
  year         = "2018",
}

##### Other

@misc{speechbrain,
  title={{SpeechBrain}: A General-Purpose Speech Toolkit},
  author={Mirco Ravanelli and Titouan Parcollet and Peter Plantinga and Aku Rouhe and Samuele Cornell and Loren Lugosch and Cem Subakan and Nauman Dawalatabad and Abdelwahab Heba and Jianyuan Zhong and Ju-Chieh Chou and Sung-Lin Yeh and Szu-Wei Fu and Chien-Feng Liao and Elena Rastorgueva and François Grondin and William Aris and Hwidong Na and Yan Gao and Renato De Mori and Yoshua Bengio},
  year={2021},
  eprint={2106.04624},
  archivePrefix={arXiv},
  primaryClass={eess.AS},
  note={arXiv:2106.04624}
}